In [ ]:
!pip install langchain-community
!pip install langchain-huggingface
!pip install groq
!pip install -U langchain-ibm ibm-watsonx-ai python-dotenv
!pip install streamlit


!pip install langchain-core
!pip install langchain_text_splitters
!pip install pypdf
!pip install langchain-chroma


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data_db.zip to Data_db.zip


In [ ]:
!unzip Data_db.zip -d docs

Archive:  Data_db.zip
   creating: docs/Data_db/
  inflating: docs/Data_db/B2C Courier Returns RTO.pdf  
  inflating: docs/Data_db/B2C Reverse PickupsDTO.pdf  
  inflating: docs/Data_db/delhivery_privacy_policy.pdf  
  inflating: docs/Data_db/logistics_data.csv  
  inflating: docs/Data_db/Lost  Damaged Claims.pdf  
  inflating: docs/Data_db/Pickup Locations.pdf  


In [ ]:
%%writefile friday.py
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
import re
import os
import pandas as pd
from datetime import datetime
os.environ["GROQ_API_KEY"] = "API-KEY"
from dotenv import load_dotenv
from groq import Groq
import streamlit as st


#  error indicator
if "last_error" not in st.session_state:
    st.session_state["last_error"] = None

#  quick helper to set error
def set_last_error(tag: str, exc: Exception):
    st.session_state["last_error"] = f"{tag}: {type(exc).__name__}: {str(exc)}"


# -------------------------
# Config
# -------------------------
load_dotenv()  # expects .env with GROQ_API_KEY

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
INDEX_DIR = "chroma_index"
DOCS_FOLDER = "docs/Data_db"

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY) if GROQ_API_KEY else None

vectorestore = None  # module-level cache


def build_chroma_index(
    docs_folder: str = DOCS_FOLDER,
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    embed_model: str = EMBED_MODEL,
):
    docs_path = os.path.abspath(docs_folder)
    if not os.path.exists(docs_path):
        raise FileNotFoundError(
            f"Docs folder not found: {docs_path}. Put your PDFs into '{docs_folder}'."
        )

    pdf_paths = [
        os.path.join(docs_path, f)
        for f in os.listdir(docs_path)
        if f.lower().endswith(".pdf")
    ]
    if not pdf_paths:
        raise FileNotFoundError(
            f"No PDF files found in {docs_path}. Place your PDFs there and retry."
        )

    pages = []
    for p in pdf_paths:
        loader = PyPDFLoader(p)
        pages.extend(loader.load())

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    split_docs = splitter.split_documents(pages)

    embeddings = HuggingFaceEmbeddings(model_name=embed_model)
    db = Chroma.from_documents(split_docs, embeddings, persist_directory=INDEX_DIR)
    try:
        db.persist()
    except Exception:
        pass
    return db


def load_chroma_index(embed_model: str = EMBED_MODEL):
    embeddings = HuggingFaceEmbeddings(model_name=embed_model)
    if os.path.isdir(INDEX_DIR) and os.listdir(INDEX_DIR):
        try:
            return Chroma(persist_directory=INDEX_DIR, embedding_function=embeddings)
        except Exception as e:
            print("Failed to load chroma index:", e)
    return None


def build_or_load_index(docs_folder: str = DOCS_FOLDER):
    existing = load_chroma_index()
    if existing:
        return existing
    return build_chroma_index(docs_folder=docs_folder)


PROMPT_TEMPLATE = """You are FRIDAY — a helpful, concise Delhivery customer-support assistant.

RULES:
1. You ONLY handle Delhivery shipments.
2. If the user reports a problem (delay, damage, lost, incorrect address, delivery failure, etc.), respond with an ISSUE SUPPORT FLOW:
   - Ask polite clarification from the user.
   - Provide possible reasons.
   - Suggest next steps.
   - Never ask which courier service; it's always Delhivery.
3. If the user asks for tracking but does not provide an Order ID, ask ONLY for the Delhivery tracking ID.
4. If an Order ID (ORDxxxx) is detected, DO NOT ask questions — system will fetch CSV details.
5. If CONTEXT is "NO_CONTEXT_FOUND", ignore it and answer normally using your own knowledge.
6. If CONTEXT is valid document text, answer ONLY from the context.

ISSUE SUPPORT FLOW EXAMPLES:
- If user says "my parcel is delayed":
    → Apologize briefly, tell possible causes, show typical delays, ask if they want an escalation.
- If user says "package damaged":
    → Apologize, ask for description or images, explain claims process.
- If user says "wrong address":
    → Provide Delhivery address-change steps.
- If user says "lost":
    → Explain investigation timeline and escalation.

CONTEXT:
{context}

Question: {question}
"""


prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)


def llm_reply(prompt_text: str, model_name: str, temperature: float) -> str:
    if client is None:
        return "Groq client is not configured. Set GROQ_API_KEY in your .env file."

    messages = [
        {"role": "system", "content": "You are a helpful, concise assistant."},
        {"role": "user", "content": prompt_text},
    ]
    resp = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=temperature,
        max_tokens=600,
    )
    return resp.choices[0].message.content.strip()

# -------------------------
# Streamlit layout
# -------------------------
st.set_page_config(page_title="F.R.I.D.A.Y", page_icon="🤖")
st.title("F.R.I.D.A.Y")

if "messages" not in st.session_state:
    st.session_state["messages"] = []

with st.sidebar:
    st.title("Settings")
    st.markdown("### Settings")
    model = st.selectbox(
        "Model",
        [
            "llama-3.1-8b-instant",
            "mixtral-8x7b-32768",
            "gemma2-9b-it",
        ],
        index=0,
    )

    temperature = st.slider("Creativity (temperature)", 0.0, 1.0, 0.3, 0.05)
    top_k = st.slider("Retrieval top-k", 1, 8, 4)

    if st.button("Clear chat"):
        st.session_state["messages"] = []
        st.rerun()

    with st.expander("Admin: last error (click to view)"):
        if st.session_state.get("last_error"):
            st.error(st.session_state["last_error"])
        else:
            st.write("No recent errors.")
# show history
for message in st.session_state["messages"]:
    st.chat_message(message["role"]).markdown(message["content"])



# -------------------------
# MAIN CHAT INPUT
# -------------------------


user_input = st.chat_input("Enter your question here:")
if user_input:
    st.chat_message("user").markdown(user_input)
    st.session_state["messages"].append(
        {"role": "user", "content": user_input}
    )

    # Initialize state
    answer = None
    order_lookup_done = False
    retrieved_docs = []

    # -------------------------
    # Build vectorstore (only once)
    # -------------------------
    if not st.session_state.get("vectorestore_built", False):
        with st.spinner("Loading knowledge base..."):
            try:
                st.session_state.vectorestore = build_or_load_index(DOCS_FOLDER)
            except Exception as e:
                st.session_state.vectorestore = None
                set_last_error("Vectorstore build failed", e)
            finally:
                st.session_state.vectorestore_built = True

    # -------------------------
    # ISSUE DETECTION (EARLY)
    # -------------------------
    issue_keywords = [
        "delay", "delayed", "late", "damaged", "broken",
        "torn", "wet", "leaking", "lost", "missing",
        "wrong address", "incorrect address", "failed delivery",
        "return to origin", "no update",
    ]

    lower_text = user_input.lower()
    issue_detected = any(word in lower_text for word in issue_keywords)
    if issue_detected:
        issue_prompt =f"""
        You are FRIDAY, a Delhivery customer-support assistant.

The user is reporting an issue: "{user_input}"

Write a short, polite, accurate explanation or apology that matches this exact issue.
- If it's damaged → talk only about damage.
- If it's lost → talk only about lost parcels.
- If it's delayed → talk only about delays.
- If it's wrong address → talk only about address issues.
Keep it human, simple, and empathetic.
"""

        try:
            issue_response = llm_reply(issue_prompt, model, temperature)
        except Exception as e:
            issue_response = "I'm sorry you're facing an issue with your shipment."

        # Always append support email
        issue_response += (
            "\n\nTo help you resolve this quickly, please write to "
            "**help@delhivery.com** with your Order ID and a short description of the problem."
        )

        answer = issue_response
        order_lookup_done = True

    # Extract Order ID
    m = re.search(r"\bORD(\d{3,6})\b", user_input, flags=re.IGNORECASE)
    order_id = f"ORD{m.group(1)}" if m else None

    # -------------------------
    # CSV Lookup (tracking)
    # -------------------------
    LOGISTICS_CSV = os.path.join(DOCS_FOLDER, "logistics_data.csv")
    if "logistics_df" not in st.session_state:
        try:
            st.session_state["logistics_df"] = pd.read_csv(LOGISTICS_CSV)
        except Exception as e:
            st.session_state["logistics_df"] = None
            set_last_error("CSV load failed", e)

    logistics_df = st.session_state["logistics_df"]

    if not order_lookup_done:
      tracking_keywords=[
          "where is my parcel",
          "where is my order",
          "track my order",
          "track parcel",
          "track my parcel",
          "parcel status",
          "order status",
          "tracking status",]
      tracking_detected = any(kw in lower_text for kw in tracking_keywords)

      if order_id and logistics_df is not None:
        row = logistics_df[logistics_df["Order_ID"].str.upper() == order_id.upper()]
        if not row.empty:
            r = row.iloc[0]
            answer = (
                f"**Tracking update for {order_id}:**\n\n"
                f"- Status: **{r['Shipment_Status']}**\n"
                f"- Pickup: {r['Pickup_City']}\n"
                f"- Destination: {r['Delivery_City']}\n"
                f"- Current Location: {r['Current_Location']}\n"
                f"- Expected Delivery: {r['Expected_Delivery_Date']}\n"
            )
            order_lookup_done = True
        else:
            answer = f"Order ID {order_id} not found in system."
            order_lookup_done = True

    # -------------------------
    # RAG Fallback
    # -------------------------
    if not order_lookup_done:
        vectorestore = st.session_state.get("vectorestore", None)

        if vectorestore is not None:
            try:
                retrieved_docs = vectorestore.similarity_search(user_input, k=top_k)
            except Exception as e:
                set_last_error("Similarity Search Failed", e)
                retrieved_docs=[]
        else:
          retrieved_docs=[]
        if retrieved_docs:
            parts = []
            for d in retrieved_docs[:top_k]:
                content = getattr(d, "page_content", None) or getattr(d, "content", None) or ""
                parts.append(str(content).replace("\n", " ")[:1500])
            context_text = "\n\n---\n\n".join(parts)
        else:
            context_text = "NO_CONTEXT_FOUND"

        final_prompt_text = prompt_template.format(
            context=context_text, question=user_input
        )

        try:
            answer = llm_reply(final_prompt_text, model, temperature)
        except Exception as e:
            answer = f"Error calling LLM: {e}"
            set_last_error("LLM call failed", e)

    # -------------------------
    # Show final answer
    # -------------------------
    with st.chat_message("assistant"):
        st.markdown(answer)

    st.session_state["messages"].append({"role": "assistant", "content": answer})

Writing friday.py


In [ ]:
! pip install streamlit -q


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.138.70.125


In [ ]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 5s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [ ]:

!streamlit run friday.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴

⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://floppy-cups-appear.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.70.125:8501

2025-11-21 09:29:15.343951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763717355.371918    1473 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763717355.380669    1473 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763717355.402841    1473 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17637173